In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import cv2
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [2]:
root_folder = '/Users/namanparuthi/Downloads/Rock Paper Scissors SXSW'


In [3]:
# Define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data_folder, labels, transform=None):
        self.data_folder = data_folder
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data_folder)

    def __getitem__(self, idx):
        img_name = self.data_folder[idx]
        image = Image.open(img_name)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
labels_df = pd.read_csv("/Users/namanparuthi/Downloads/Rock Paper Scissors SXSW/train/_annotations.csv")
labels_dictionary = {'Paper': 0, 'Rock': 1, 'Scissors': 2}
labels_df['class'] = labels_df['class'].map(labels_dictionary)
labels = labels_df['class']
image_paths = [os.path.join(root_folder, 'train', filename) for filename in labels_df['filename']]

In [5]:
labels_df


,filename,width,height,class,xmin,ymin,xmax,ymax
0,zoom_tests_RockPaperScissors-mohamed_mp4-67_jp...,640,640,1,12,0,141,250
1,egohands-public-1625070649053_png_jpg.rf.000a5...,640,640,1,153,407,485,637
2,zoom_tests_RockPaperScissors-mohamed_mp4-111_j...,640,640,0,0,227,233,640
3,IMG_5636_MOV-61_jpg.rf.003476f680b71b74f3764a1...,640,640,1,28,367,175,467
4,IMG_5636_MOV-61_jpg.rf.003476f680b71b74f3764a1...,640,640,0,281,333,556,452
...,...,...,...,...,...,...,...,...
7816,IMG_5567_mp4-163_jpg.rf.c7c964e5f3334f60c8f6f6...,640,640,0,416,305,623,430
7817,Screen-Shot-2022-02-08-at-1-00-58-PM_png.rf.c7...,640,640,2,461,179,640,477
7818,IMG_5567_mp4-89_jpg.rf.c7b9f669eca26311965b115...,640,640,0,220,381,435,519
7819,IMG_7043_MOV-176_jpg.rf.c7cc43070427a0b1b23af9...,640,640,1,0,226,520,526


In [6]:
print(len(labels_df))


7821


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(image_paths, labels, test_size=0.2, random_state=42)


In [8]:
print(len(X_train), len(Y_train))

6256 6256


In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [10]:
train_dataset = CustomDataset(data_folder=X_train, labels=Y_train, transform=transform)
test_dataset = CustomDataset(data_folder=X_test, labels=Y_test, transform=transform)


In [11]:
batch_size = 32  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [12]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [13]:
num_classes = 3  # Number of classes (Rock, Paper, Scissors)
model = CNNModel(num_classes)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
num_epochs = 10

In [21]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for images,labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_loader)}')

KeyError: 3541

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

In [ ]:
torch.save(model.state_dict(), 'rock_paper_scissors_cnn.pth')